<a href="https://colab.research.google.com/github/widura26/machine-learning-portfolio/blob/main/Book_recommendations_system_Collaborative_Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import drive
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import regex
import re
import numpy as np

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/MyDrive/dataset/book-dataset'

In [ ]:
books = pd.read_csv(f'{file_path}/books.csv')
tags = pd.read_csv(f'{file_path}/tags.csv')
book_tags = pd.read_csv(f'{file_path}/book_tags.csv')
ratings = pd.read_csv(f'{file_path}/ratings.csv')
to_read = pd.read_csv(f'{file_path}/to_read.csv')

In [ ]:
books_ratings = pd.merge(books, ratings, on="book_id", how="right")

In [ ]:
books_ratings.isna().sum()

,0
id,902055
book_id,0
best_book_id,902055
work_id,902055
books_count,902055
isbn,902934
isbn13,902453
authors,902055
original_publication_year,902155
original_title,903836


In [ ]:
user_item_matrix = books_ratings.pivot_table(
    index="user_id",
    columns="book_id",
    values="rating",
    fill_value=0
)

In [ ]:
user_item_matrix

book_id,1,2,3,4,5,6,7,8,9,10,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53420,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53421,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53422,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
item_similarity = cosine_similarity(user_item_matrix.T)

In [ ]:
item_similarity_df = pd.DataFrame(
    item_similarity,
    index=user_item_matrix.columns,
    columns=user_item_matrix.columns
)

In [ ]:
item_similarity_df

book_id,1,2,3,4,5,6,7,8,9,10,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
book_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.555526,0.509152,0.464656,0.480233,0.244545,0.417384,0.389550,0.472808,0.401206,...,0.011966,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
2,0.555526,1.000000,0.520216,0.610741,0.587944,0.171646,0.572666,0.495658,0.488789,0.546173,...,0.000000,0.000000,0.000000,0.000000,0.004247,0.0,0.000000,0.000000,0.0,0.006921
3,0.509152,0.520216,1.000000,0.359108,0.344423,0.289692,0.367322,0.472219,0.345934,0.513706,...,0.009096,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
4,0.464656,0.610741,0.359108,1.000000,0.703335,0.112823,0.517456,0.630009,0.453919,0.574839,...,0.011425,0.000000,0.000000,0.000000,0.016234,0.0,0.000000,0.000000,0.0,0.000000
5,0.480233,0.587944,0.344423,0.703335,1.000000,0.179174,0.540620,0.633974,0.395894,0.553986,...,0.000000,0.000000,0.000000,0.000000,0.015976,0.0,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.010111,0.000000,0.004683,0.000000,1.0,0.000000,0.000000,0.0,0.000000
9997,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,0.022634,0.0,0.011805
9998,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.014739,0.009068,0.000000,0.008610,0.0,0.022634,1.000000,0.0,0.000000


In [ ]:
def predict_rating_item_based(user_id, book_id):
    user_ratings = user_item_matrix.loc[user_id]
    sim_scores = item_similarity_df[book_id]
    rated_books = user_ratings[user_ratings > 0].index

    numerator = np.dot(user_ratings[rated_books], sim_scores[rated_books])
    denominator = sim_scores[rated_books].sum()

    if denominator == 0:
        return 0
    return numerator / denominator

In [ ]:
books_ratings[(books_ratings['user_id'] == 1) & (books_ratings['book_id'] == 1)]
books_ratings[books_ratings['user_id'] == 1][['user_id','book_id', 'rating']]

,user_id,book_id,rating
117889,1,1180,4
488112,1,4893,3
625717,1,6285,4


In [ ]:
predict_rating_item_based(1, 6285)

np.float64(3.9925373259427928)

In [ ]:
def recommend_movies_item_based(user_id, top_n=5):
    user_ratings = user_item_matrix.loc[user_id]
    books_not_rated = user_ratings[user_ratings == 0].index

    predictions = {book: predict_rating_item_based(user_id, book) for book in books_not_rated}
    recommended = sorted(predictions.items(), key=lambda x: x[1], reverse=True)[:top_n]

    rec_df = pd.DataFrame(recommended, columns=['book_id', 'predicted_rating'])
    rec_df = rec_df.merge(books, on='book_id', how='left')

    return rec_df[['book_id', 'title', 'predicted_rating']]

recommend_movies_item_based(2)

,book_id,title,predicted_rating
0,3949,NaN,5.0
1,6893,NaN,5.0
2,9203,NaN,5.0
3,1085,NaN,5.0
4,1125,NaN,5.0
